<a href="https://colab.research.google.com/github/sumedh151/pytorch-transformer/blob/main/run_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python /content/drive/MyDrive/Colab\ Notebooks/attention\ is\ all\ you\ need/main.py

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
# https://www.youtube.com/watch?v=ISNdQcPhsts

import torch
import torch.nn as nn
import math
# from config import config
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from torch.utils.data import Dataset, DataLoader

In [ ]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

dataset = load_dataset("cfilt/iitb-english-hindi", split='train')

print(dataset[0])
print(dataset[0]['translation'])
print(dataset[0]['translation']['en'])
print(dataset[0]['translation']['hi'])

if __name__ == '__main__':
    print('hello world')
    # print(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}
{'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}
Give your application an accessibility workout
अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
hello world


In [ ]:
dataset

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

dataset = load_dataset("cfilt/iitb-english-hindi")

# def get_texts(dataset, lang):
def get_texts(dataset):
    for example in dataset['train']['translation']:
        # yield example[lang]
        yield example['en']
        yield example['hi']

tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.ByteLevel()

trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

# tokenizer.train_from_iterator(get_texts(dataset, 'en'), trainer=trainer)
# tokenizer.train_from_iterator(get_texts(dataset, 'hi'), trainer=trainer)
tokenizer.train_from_iterator(get_texts(dataset), trainer=trainer)

tokenizer.save("iitb-english-hindi-tokenizer.json")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
# Encode some text
max_length = 50
encoding = tokenizer.encode("This is a test sentence.")
print(encoding.tokens)

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġsentence', '.']


In [ ]:
tokenizer.get_vocab_size()

30000

In [ ]:
tokenizer.token_to_id("<unk>")

3

In [ ]:
tokenizer.enable_padding(pad_id=1, pad_token="<pad>")
output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])
print(output[0].tokens)
print(output[0].attention_mask)
print(output[1].tokens)
print(output[1].attention_mask)

['Hell', 'o', ',', 'Ġy', "'", 'all', '!', '<pad>']
[1, 1, 1, 1, 1, 1, 1, 0]
['How', 'Ġare', 'Ġyou', 'Ġ', 'Ł', 'ĺ', 'ģ', 'Ġ?']
[1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# Load the trained tokenizer
tokenizer = Tokenizer.from_file("iitb-english-hindi-tokenizer.json")

# Encode some text
encoding = tokenizer.encode("This is a test sentence.")
print(encoding.tokens)
print(encoding.ids)

# Decode back to text
decoded_text = tokenizer.decode(encoding.ids)
print(decoded_text)

['This', 'Ġis', 'Ġa', 'Ġtest', 'Ġsentence', '.']
[874, 317, 220, 2527, 9713, 18]
This is a test sentence.


In [ ]:
dataset['train']['translation'][:10]

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

In [ ]:
len(dataset['train']['translation'][0]['hi'])

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
import tqdm
class CustomTranslationDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=64, split='train'):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.split = split
        self.max_length = max_length
        # self.tokenized_data, self.attention_masks = self.tokenize_data()
        self.tokenized_data = self.tokenize_data()

    def tokenize_data(self):
        tokens = []
        attention_masks = []
        tokenized_data = []
        c=0
        for example in tqdm.tqdm(self.dataset[self.split]['translation']):
        # for example in self.dataset[self.split]['translation']:
            en_text = example['en']
            hi_text = example['hi']

            en_tokens = self.tokenizer.encode(en_text).ids[:self.max_length - 2]  # -2 to account for <s> and </s>
            hi_tokens = self.tokenizer.encode(hi_text).ids[:self.max_length - 2]

            en_tokens = [self.tokenizer.token_to_id("<s>")] + en_tokens + [self.tokenizer.token_to_id("</s>")]
            hi_tokens = [self.tokenizer.token_to_id("<s>")] + hi_tokens + [self.tokenizer.token_to_id("</s>")]

            len_en = len(en_tokens)
            len_h1 = len(hi_tokens)

            en_padding_length = self.max_length - len_en
            hi_padding_length = self.max_length - len_h1

            en_tokens = en_tokens + [self.tokenizer.token_to_id("<pad>")] * en_padding_length
            hi_tokens = hi_tokens + [self.tokenizer.token_to_id("<pad>")] * hi_padding_length

            en_attention_mask = [1] * len_en + [0] * en_padding_length
            hi_attention_mask = [1] * len_h1 + [0] * hi_padding_length

            # tokens.append((en_tokens, hi_tokens))
            # attention_masks.append((en_attention_mask, hi_attention_mask))
            tokenized_data.append((en_tokens, hi_tokens, en_attention_mask, hi_attention_mask))
            # c+=0
            # if c==9:
            #     break
        # return tokens, attention_masks
        return tokenized_data

    def __getitem__(self, idx):
        # return self.tokenized_data[idx] , self.attention_masks[idx]
        return self.tokenized_data[idx]

    def __len__(self):
        return len(self.tokenized_data)

In [ ]:
ds = CustomTranslationDataset(dataset, tokenizer, max_length=64)

100%|██████████| 1659083/1659083 [09:20<00:00, 2961.51it/s]


In [ ]:
c=0
for i in ds:
    print(i)
    c+=1
    if c==4:
        break

([0, 12879, 593, 2865, 388, 20185, 872, 678, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1435, 207, 421, 230, 237, 208, 721, 217, 244, 210, 217, 543, 891, 679, 213, 575, 205, 252, 208, 219, 205, 219, 205, 223, 210, 205, 277, 205, 316, 271, 246, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
([0, 3828, 2199, 15705, 13974, 2803, 19351, 18660, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
train_data_loader = DataLoader(ds, batch_size=4, shuffle=False)
for i in train_data_loader:
    print(i)
    print(len(i))
    break

[[tensor([0, 0, 0, 0]), tensor([12879,  3828,   350,   350]), tensor([ 593, 2199, 4508, 4508]), tensor([ 2865, 15705,  6824,  6824]), tensor([  388, 13974, 11074, 11074]), tensor([20185,  2803,   331,   331]), tensor([  872, 19351,   228,   228]), tensor([  678, 18660,  8304,  3070]), tensor([   2,    2, 9578, 9578]), tensor([1, 1, 2, 2]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1])

In [ ]:
c = 0
max_en = 0
max_hi = 0
for i in tqdm.tqdm(ds):
    # print(tokenizer.decode(i[0][0]))
    # print(tokenizer.decode(i[0][1]))
    # c+=1
    # if c ==100:
    #     break
    print(i[1][0])
    print(sum(i[1][0]))
    print(i[1][1])
    print(sum(i[1][1]))
    break
    max_en = max(max_en , sum(i[1][0]))
    max_hi = max(max_hi , sum(i[1][1]))
print(max_en , max_hi)

  0%|          | 0/1659083 [00:00<?, ?it/s]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
33
0 0


In [ ]:
import gc
gc.collect()

353

In [ ]:
trans_data_loader = DataLoader(ds)

In [ ]:
for i in ds:
    print(tokenizer.decode(i[0]))
    print(len(tokenizer.decode(i[1])))
    print(len(i[1]))
    break

Give your application an accessibility workout
47
31


In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

dataset = load_dataset("cfilt/iitb-english-hindi")

def get_texts(dataset, lang):
    for example in dataset['train']['translation']:
        yield example[lang]

tokenizer_en = Tokenizer(models.BPE())
tokenizer_en.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer_en.decoder = decoders.ByteLevel()
tokenizer_en.post_processor = processors.ByteLevel()
trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer_en.train_from_iterator(get_texts(dataset, 'en'), trainer=trainer)

tokenizer_en.save("iitb-english-hindi-tokenizer_en.json")

tokenizer_hi = Tokenizer(models.BPE())
tokenizer_hi.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer_hi.decoder = decoders.ByteLevel()
tokenizer_hi.post_processor = processors.ByteLevel()
trainer = trainers.BpeTrainer(special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
tokenizer_hi.train_from_iterator(get_texts(dataset, 'hi'), trainer=trainer)

tokenizer_hi.save("iitb-english-hindi-tokenizer_hi.json")

In [ ]:
tokenizer.get_vocab_size()

30000

In [ ]:
max_len=5000
# position = torch.arange(max_len).unsqueeze(1)
position = torch.zeros((max_len,10))
position = torch.zeros((max_len,10)).unsqueeze(1)
position.shape

torch.Size([5000, 1, 10])

In [ ]:
m = nn.Softmax(dim=-1)
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

tensor([[-2.4445, -0.3239,  1.1292],
        [-2.4652, -1.6679, -0.8715]])
tensor([[0.0222, 0.1853, 0.7925],
        [0.1228, 0.2726, 0.6046]])


In [ ]:
class Encoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Encoder, self).__init__()
        # self.query_weight = nn.Parameter(torch.randn(d, dk), requires_grad=True)
        # self.key_weight   = nn.Parameter(torch.randn(d, dk), requires_grad=True)
        # self.value_weight = nn.Parameter(torch.randn(d, dv), requires_grad=True)
        self.query_layer = nn.Linear(d,dk)
        self.key_layer = nn.Linear(d,dk)
        self.value_layer = nn.Linear(d,dv)
        self.d = d
        self.dk = dk
        self.dv = dv
        self.h = h
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ffo = nn.Linear(self.h * self.dv , self.d)
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def multihead_attention(self, query, key, value):
        # same layers???
        head_outputs = []
        for _ in range(self.h):
            head_output = self.self_attention(query, key, value)
            head_outputs.append(head_output)
        multihead = torch.concat(head_outputs, dim=-1)
        return self.ffo(multihead)

    def self_attention(self, query, key, value):
        qt = torch.bmm(query, torch.transpose(key,1,2))
        attn_weights = nn.Softmax(dim=-1)(qt/np.sqrt(self.dk))
        self_attn = torch.bmm(qt, value)
        return self_attn

    def forward(self,x):
        # query = torch.matmul(x, self.query_weight)
        # key   = torch.matmul(x, self.key_weight)
        # value = torch.matmul(x, self.value_weight)
        query = self.query_layer(x)
        key   = self.key_layer(x)
        value = self.value_layer(x)

        multihead_attn = self.multihead_attention(query, key, value)
        resi1 = multihead_attn + x
        norm1 = self.layer_norm1(resi1)
        feed_forw1 = self.ff1(norm1)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi2 = feed_forw2 + norm1
        norm2 = self.layer_norm2(resi2)
        return norm2

In [ ]:
from torch import nn
import numpy as np

In [ ]:
# output = tokenizer.encode_batch(["Hello, y'all!", "How are you 😁 ?"])
output = tokenizer.encode_batch([["Hello, y'all!", "Hello, y'all!"] , ["Hello, y'all!", "Hello, y'all!"]])
# output = tokenizer.encode_batch(["Hello, y'all!", "Hello, y'all!"])
output = [x.ids for x in output]
print(output)
torch.Tensor(output).shape

[[14823, 83, 16, 325, 11, 465, 5, 14823, 83, 16, 325, 11, 465, 5], [14823, 83, 16, 325, 11, 465, 5, 14823, 83, 16, 325, 11, 465, 5]]


torch.Size([2, 14])

In [ ]:
del dataset

In [ ]:
def mask_upper_tri_matrix(matrix):
    upper_triangle_mask = torch.triu(torch.ones_like(matrix, dtype=torch.bool), diagonal=1)
    return matrix.masked_fill(upper_triangle_mask, float('-inf'))

mask_upper_tri_matrix(torch.randn(5, 5))

tensor([[-0.7800,    -inf,    -inf,    -inf,    -inf],
        [ 0.4139,  0.8619,    -inf,    -inf,    -inf],
        [ 0.0748,  1.4514, -0.8199,    -inf,    -inf],
        [ 0.0096,  0.4620,  0.9776,  1.5842,    -inf],
        [ 0.3352,  0.5023,  1.7969, -1.3255,  0.2893]])

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self) -> None:
        super(PositionalEncoding,self).__init__()

    def forward(self,x):
        return x

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, d, dk, dv, mask):
        super(SelfAttention, self).__init__()
        self.query_layer = nn.Linear(d,dk)
        self.key_layer = nn.Linear(d,dk)
        self.value_layer = nn.Linear(d,dv)
        self.d = d
        self.dk = dk
        self.dv = dv
        self.mask = mask

    def forward(self, x, enc_x=None):
        # print("x shp in enc",x.shape)
        query = self.query_layer(x)
        if enc_x is not None:
            key   = self.key_layer(enc_x)
            value = self.value_layer(enc_x)
        else:
            key   = self.key_layer(x)
            value = self.value_layer(x)
        qt = torch.bmm(query, torch.transpose(key,1,2))
        if self.mask:
            qt = mask_upper_tri_matrix(qt)
        attn_weights = nn.Softmax(dim=-1)(qt/np.sqrt(self.dk))
        self_attn = torch.bmm(qt, value)
        return self_attn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, d, dk, dv, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.h = h
        self.d = d
        self.dk = dk
        self.dv = dv
        self.ffo = nn.Linear(self.h * self.dv , self.d)
        self.mask = mask

    def forward(self,x,enc_x=None):
        head_outputs = []
        for _ in range(self.h):
            head_output = SelfAttention(self.d , self.dk , self.dv , self.mask)(x , enc_x)
            head_outputs.append(head_output)
        multihead = torch.concat(head_outputs, dim=-1)
        return self.ffo(multihead)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Encoder, self).__init__()
        self.d = d
        self.dk = dk
        self.dv = dv
        self.h = h
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def forward(self,x):
        multihead_attn = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=False)(x)
        resi1 = multihead_attn + x
        norm1 = self.layer_norm1(resi1)
        feed_forw1 = self.ff1(norm1)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi2 = feed_forw2 + norm1
        norm2 = self.layer_norm2(resi2)
        return norm2

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d, dk, dv, h) -> None:
        super(Decoder, self).__init__()
        self.h = h
        self.d = d
        self.dk = dk
        self.dv = dv
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.layer_norm3 = nn.LayerNorm(d)
        self.ffo_out = 2048
        self.ff1 = nn.Linear(self.d , self.ffo_out)
        self.ff2 = nn.Linear(self.ffo_out , self.d)
        self.relu = nn.ReLU()

    def forward(self,x,enc_x):
        multihead_attn1 = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=True)(x)
        resi1 = multihead_attn1 + x
        norm1 = self.layer_norm1(resi1)
        multihead_attn2 = MultiHeadAttention(self.h, self.d, self.dk, self.dv, mask=False)(x,enc_x)
        resi2 = multihead_attn2 + norm1
        norm2 = self.layer_norm2(resi2)
        feed_forw1 = self.ff1(norm2)
        feed_forw1 = self.relu(feed_forw1)
        feed_forw2 = self.ff2(feed_forw1)
        resi3 = feed_forw2 + norm2
        norm3 = self.layer_norm2(resi3)
        return norm3

In [ ]:
class Transformer(nn.Module):
    def __init__(self, tokenizer, nx=6) -> None:
        super(Transformer, self).__init__()
        self.tokenizer = tokenizer
        self.nx = nx
        self.d = 256
        self.dk = 512
        self.dv = 512
        self.h = 8
        self.inp_embedding = nn.Embedding(self.tokenizer.get_vocab_size(), self.d, max_norm=True)
        self.out_embedding = nn.Embedding(self.tokenizer.get_vocab_size(), self.d, max_norm=True)
        self.positional_encoding = PositionalEncoding()
        self.encoder = Encoder(d = self.d, dk = self.dk, dv = self.dv, h = self.h)
        # self.encoders = nn.ModuleList([Encoder(d=self.d, dk=self.dk, dv=self.dv, h=self.h) for _ in range(self.nx)])
        self.decoder = Decoder(d = self.d, dk = self.dk, dv = self.dv, h = self.h)
        # self.decoders = nn.ModuleList([Decoder(d=self.d, dk=self.dk, dv=self.dv, h=self.h) for _ in range(self.nx)])
        self.projection = nn.Linear(self.d, self.tokenizer.get_vocab_size())

    def forward(self, input_tokens):
        # input_tokens = torch.tensor(input_tokens, dtype=torch.int64)
        x = self.inp_embedding(input_tokens)
        x = self.positional_encoding(x)
        for _ in range(self.nx):
            x = self.encoder(x)
        # for encoder in self.encoders:
        #     x = encoder(x)

        dec_x = self.out_embedding(input_tokens)
        for _ in range(self.nx):
            dec_x = self.decoder(dec_x, x)
        # for decoder in self.decoders:
        #     dec_x = decoder(dec_x, x)
        out = self.projection(dec_x)
        return out

In [ ]:
# model = Transformer(tokenizer)(torch.LongTensor(output))
model = Transformer(tokenizer)(torch.tensor(output, dtype=torch.int64))

torch.Size([2, 7, 256])


In [ ]:
train_data_loader = DataLoader(ds, batch_size=4, shuffle=False)
for i in train_data_loader:
    print(i)
    break

[[[tensor([0, 0, 0, 0]), tensor([12879,  3828,   350,   350]), tensor([ 593, 2199, 4508, 4508]), tensor([ 2865, 15705,  6824,  6824]), tensor([  388, 13974, 11074, 11074]), tensor([20185,  2803,   331,   331]), tensor([  872, 19351,   228,   228]), tensor([  678, 18660,  8304,  3070]), tensor([   2,    2, 9578, 9578]), tensor([1, 1, 2, 2]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]), tensor([1, 1, 1, 1]

In [ ]:
from torchsummary import summary
model = Transformer(tokenizer)
print(type(model))
summary(model , input_size=(100,2,7))

<class '__main__.Transformer'>


<ipython-input-164-b6569de0eef8>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tokens = torch.tensor(input_tokens, dtype=torch.int64)


RuntimeError: batch1 must be a 3D tensor

In [ ]:
output

[[14823, 83, 16, 325, 11, 465, 5], [14823, 83, 16, 325, 11, 465, 5]]

In [ ]:
torch.LongTensor(output)

tensor([[14823,    83,    16,   325,    11,   465,     5],
        [14823,    83,    16,   325,    11,   465,     5]])

In [ ]:
np.array(output, dtype=np.int32)

array([[14823,    83,    16,   325,    11,   465,     5],
       [14823,    83,    16,   325,    11,   465,     5]], dtype=int32)

In [ ]:
nn.Linear(10,10)

Linear(in_features=10, out_features=10, bias=True)

In [ ]:
l = nn.Linear(10,20)

In [ ]:
x = torch.randn(128, 32,10)
x.shape

torch.Size([128, 32, 10])

In [ ]:
l(x).shape

torch.Size([128, 32, 20])

In [ ]:
l(x).size()

torch.Size([32, 20])

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 10)  # Adjust 16 * 16 according to your input size

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 16 * 16)  # Adjust 16 * 16 according to your input size
        x = self.fc1(x)
        return x

In [ ]:
def model_summary(model, input_size):
    summary(model, input_size=input_size)

In [ ]:
model = MyModel()
input_size = (3, 32, 32)  # Example input size (channels, height, width)
model_summary(model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
         MaxPool2d-3           [-1, 64, 16, 16]               0
            Linear-4                   [-1, 10]         163,850
Total params: 165,642
Trainable params: 165,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.13
Params size (MB): 0.63
Estimated Total Size (MB): 1.77
----------------------------------------------------------------
